# 3.3 线性回归的简洁实现

In [2]:
import torch
from torch import nn
import numpy as np
torch.manual_seed(1)

print(torch.__version__)
torch.set_default_tensor_type('torch.FloatTensor')

2.5.1


D:\Anaconda\envs\Pytorch_GPU\Lib\site-packages\torch\__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


## 3.3.1 生成数据集

In [3]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
# features = torch.tensor(np.random.normal(0,1,(num_examples, num_inputs)), dtype=torch.float)
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
print(features[:10])
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

tensor([[ 1.0055,  0.9919],
        [-0.3676, -0.7837],
        [-0.5909, -0.4566],
        [ 0.7196, -0.1309],
        [ 0.4773, -0.7461],
        [-0.9321, -0.7150],
        [ 0.2805,  1.4043],
        [-0.5009,  0.4753],
        [ 1.1579,  0.3292],
        [-1.7767, -0.1681]])


## 3.3.2 读取数据

In [7]:
import torch.utils.data as Data
from torch.utils import data

batch_size = 10

# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# dataset = data.TensorDataset(features,labels)

# 把 dataset 放入 DataLoader
data_iter = Data.DataLoader(
    dataset=dataset,      # torch TensorDataset format
    batch_size=batch_size,      # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好),在训练时shuffle，测试时关闭
    num_workers=2,              # 多线程来读数据
)


AttributeError: 'DataLoader' object has no attribute 'shape'

In [6]:
for X, y in data_iter:
    print(X, '\n', y)
    break

tensor([[ 0.0295, -0.6778],
        [-1.2272,  0.6378],
        [-0.7114, -0.2982],
        [-0.2233, -0.9100],
        [ 0.3362,  0.5143],
        [-1.0095,  0.1343],
        [ 0.1935, -1.1564],
        [ 0.2241, -0.7304],
        [ 2.4348, -1.0171],
        [ 0.3801, -0.1080]]) 
 tensor([ 6.5557, -0.4136,  3.7851,  6.8536,  3.1135,  1.7264,  8.5269,  7.1339,
        12.5274,  5.3123])


## 3.3.3 定义模型

In [ ]:
class linearNet(nn.Module):
    def __int__(self, n_feature):
        super(LinearNet, self).__int__()

In [9]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)

    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构



LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [6]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [7]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.5347, 0.7057]], requires_grad=True)
Parameter containing:
tensor([0.6873], requires_grad=True)


## 3.3.4 初始化模型参数

In [8]:
from torch.nn import init

init.normal_(net[0].weight, mean=0.0, std=0.01)
init.constant_(net[0].bias, val=0.0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

In [9]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.0142, -0.0161]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


## 3.3.5 定义损失函数

In [12]:
loss = nn.MSELoss()

## 3.3.6 定义优化算法

In [11]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.03
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [12]:
# 为不同子网络设置不同的学习率
# optimizer =optim.SGD([
#                 # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#                 {'params': net.subnet1.parameters()}, # lr=0.03
#                 {'params': net.subnet2.parameters(), 'lr': 0.01}
#             ], lr=0.03)

In [13]:
# # 调整学习率
# for param_group in optimizer.param_groups:
#     param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

## 3.3.7 训练模型

In [13]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)     # 预测值
        l = loss(output, y.view(-1, 1))    # 损失函数
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()   # 反向传播，更新梯度
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000213
epoch 2, loss: 0.000028
epoch 3, loss: 0.000057


In [14]:
dense = net[0]
print(true_w, dense.weight.data)
print(true_b, dense.bias.data)

TypeError: 'LinearNet' object is not subscriptable